In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
from sklearn.datasets import make_regression
from datetime import datetime

x, y = make_regression(n_samples = 100,n_features = 5,n_targets = 1, random_state = 1)
dfx = pd.DataFrame(x, columns=['A','B','C','D','E'])
dfy = pd.DataFrame(y, columns=['label'])

# Create an integer column, a datetime column, a categorical column and a string column to demonstrate how they are processed.
dfx['B'] = (dfx['B']).astype(int)
dfx['C'] = datetime(2000,1,1) + pd.to_timedelta(dfx['C'].astype(int), unit='D')
dfx['D'] = pd.cut(dfx['D'] * 10, [-np.inf,-5,0,5,np.inf],labels=['v','w','x','y'])
dfx['E'] = pd.Series(list(' '.join(random.choice(["abc", "d", "ef", "ghi", "jkl"]) for i in range(4)) for j in range(100)))
dataset=TabularDataset(dfx)
print(dfx)

           A  B          C  D                E
0  -0.545774  0 2000-01-01  y        d jkl d d
1  -0.468674  0 2000-01-02  x     d ef ghi abc
2   1.767960  0 1999-12-31  v      jkl ef d ef
3  -0.118771  1 2000-01-01  y    d jkl abc ghi
4   0.630196  0 1999-12-31  w          d d d d
..       ... ..        ... ..              ...
95 -1.182318 -1 2000-01-01  v  jkl jkl ghi ghi
96  0.562761  0 2000-01-01  v      d abc ef ef
97 -0.797270  0 2000-01-01  w       ef ghi d d
98  0.502741  0 1999-12-31  y    abc ghi d jkl
99  2.056356  0 1999-12-30  w      ef d ghi ef

[100 rows x 5 columns]


In [2]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,0,3,NaN,946684800000000000,2000,1,1,5,2,6,0,0,0,1,1
1,-0.468674,0,2,1,946771200000000000,2000,1,2,6,5,3,1,1,1,0,3
2,1.767960,0,0,NaN,946598400000000000,1999,12,31,4,4,4,0,2,0,1,2
3,-0.118771,1,3,NaN,946684800000000000,2000,1,1,5,6,2,1,0,1,1,3
4,0.630196,0,1,NaN,946598400000000000,1999,12,31,4,0,8,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,-1,0,NaN,946684800000000000,2000,1,1,5,8,0,0,0,2,2,2
96,0.562761,0,0,NaN,946684800000000000,2000,1,1,5,4,4,1,2,0,0,2
97,-0.797270,0,1,NaN,946684800000000000,2000,1,1,5,3,5,0,1,1,0,2
98,0.502741,0,3,NaN,946598400000000000,1999,12,31,4,6,2,1,0,1,1,3


In [3]:
df = pd.concat([dfx, dfy], axis=1)
predictor = TabularPredictor(label='label')
predictor.fit(df, hyperparameters={'GBM' : {}}, feature_generator=auto_ml_pipeline_feature_generator)

No path specified. Models will be saved in: "AutogluonModels/ag-20240903_123652"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.3-Ubuntu SMP PREEMPT_DYNAMIC Tue Jul 30 17:30:19 UTC 2
CPU Count:          32
Memory Avail:       56.77 GB / 62.51 GB (90.8%)
Disk Space Avail:   681.15 GB / 843.94 GB (80.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='mediu

In [4]:
print(len(set(dfx['B'])))

5


In [5]:
dfx["B"] = dfx["B"].astype("category")
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    58102.88 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,-0.545774,1,3,NaN,946684800000000000,2000,1,1,5,2,6,0,0,0,1,1
1,-0.468674,1,2,1,946771200000000000,2000,1,2,6,5,3,1,1,1,0,3
2,1.767960,1,0,NaN,946598400000000000,1999,12,31,4,4,4,0,2,0,1,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,6,2,1,0,1,1,3
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,0,8,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,NaN,946684800000000000,2000,1,1,5,8,0,0,0,2,2,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,4,4,1,2,0,0,2
97,-0.797270,1,1,NaN,946684800000000000,2000,1,1,5,3,5,0,1,1,0,2
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,6,2,1,0,1,1,3


In [6]:
dfx.iloc[0] = np.nan
dfx.head()

,A,B,C,D,E
0,NaN,NaN,NaT,NaN,NaN
1,-0.468674,0,2000-01-02,x,d ef ghi abc
2,1.767960,0,1999-12-31,v,jkl ef d ef
3,-0.118771,1,2000-01-01,y,d jkl abc ghi
4,0.630196,0,1999-12-31,w,d d d d


In [7]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=dfx)

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    58104.43 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

,A,B,D,E,C,C.year,C.month,C.day,C.dayofweek,E.char_count,E.word_count,E.symbol_ratio.,__nlp__.abc,__nlp__.ef,__nlp__.ghi,__nlp__.jkl,__nlp__._total_
0,NaN,NaN,NaN,NaN,946687418181818240,2000,1,1,5,0,0,0,0,0,0,0,0
1,-0.468674,1,2,1,946771200000000000,2000,1,2,6,6,1,4,1,1,1,0,3
2,1.767960,1,0,NaN,946598400000000000,1999,12,31,4,5,1,5,0,2,0,1,2
3,-0.118771,2,3,NaN,946684800000000000,2000,1,1,5,7,1,3,1,0,1,1,3
4,0.630196,1,1,NaN,946598400000000000,1999,12,31,4,1,1,9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.182318,0,0,NaN,946684800000000000,2000,1,1,5,9,1,1,0,0,2,2,2
96,0.562761,1,0,NaN,946684800000000000,2000,1,1,5,5,1,5,1,2,0,0,2
97,-0.797270,1,1,NaN,946684800000000000,2000,1,1,5,4,1,6,0,1,1,0,2
98,0.502741,1,3,NaN,946598400000000000,1999,12,31,4,7,1,3,1,0,1,1,3


In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load your dataset
data = TabularDataset('/home/michael/Documenti/Milinda_Githubproject/Feartures_FIDAC_1.0.csv')

# Define the label (target column)
label = 'Label'  # Assuming the label column is named 'label'

# Split the dataset into training (80%) and testing (20%) sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data[label])

# Display the first few rows of training data
train_data.head()

# Check the unique classes in the label
print(f"Unique classes: {list(train_data[label].unique())}")

# Create and train the predictor on the training data
predictor = TabularPredictor(label=label).fit(train_data)

# Predict on the test data
y_pred = predictor.predict(test_data)
y_pred.head()  # Predictions

# Evaluate the model performance on the test data
performance = predictor.evaluate(test_data)
print("Performance on test data:", performance)

# Display the leaderboard on the test data
predictor.leaderboard(test_data)

# Print the inferred problem type and feature metadata
print("AutoGluon infers problem type is:", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

# Transform test data features (optional step to see how data is transformed)
test_data_transform = predictor.transform_features(test_data)
test_data_transform.head()

# Calculate and display feature importance
feature_importance = predictor.feature_importance(test_data)
print("Feature Importance:")
print(feature_importance)

# Display the best model
best_model = predictor.get_model_best()
print("Best model:", best_model)

# Get the best model
best_model = predictor.get_model_best()

# Predict on the test data using the best model
y_pred = predictor.predict(test_data, model=best_model)
y_true = test_data[label]

# Calculate Accuracy, Precision, Recall, and F1 Score
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print the metrics
print(f"Best Model: {best_model}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")


No path specified. Models will be saved in: "AutogluonModels/ag-20240903_165219"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.3-Ubuntu SMP PREEMPT_DYNAMIC Tue Jul 30 17:30:19 UTC 2
CPU Count:          32
Memory Avail:       56.42 GB / 62.51 GB (90.2%)
Disk Space Avail:   680.70 GB / 843.94 GB (80.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='mediu

Unique classes: [1, 0]


	0.7621	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.7586	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.9828	 = Validation score   (accuracy)
	0.27s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ...
	0.9966	 = Validation score   (accuracy)
	0.13s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	1.0	 = Validation score   (accuracy)
	0.56s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestEntr ...
	1.0	 = Validation score   (accuracy)
	0.45s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.9966	 = Validation score   (accuracy)
	0.24s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9862	 = Validation score   (accuracy)
	0.45s	 = Training   runtime
	0.08s	 = Validation

Performance on test data: {'accuracy': 0.9972375690607734, 'balanced_accuracy': 0.9979423868312758, 'mcc': 0.993773482178927, 'roc_auc': 1.0, 'f1': 0.9979381443298969, 'precision': 1.0, 'recall': 0.9958847736625515}


These features in provided data are not utilized by the predictor and will be ignored: ['Dissimilarity', 'SecondMoment', 'AverageGrayLevel']
Computing feature importance via permutation shuffling for 16 features using 362 rows with 5 shuffle sets...
	4.64s	= Expected runtime (0.93s per shuffle set)


AutoGluon infers problem type is: binary
AutoGluon identified the following types of features:
('float', []) : 15 | ['Mean', 'Variance', 'Homogeneity', 'Contrast', 'Entropy', ...]
('int', [])   :  1 | ['ImageNum']


	0.75s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:
                  importance    stddev       p_value  n  p99_high   p99_low
ImageNum            0.440331  0.015206  1.703817e-07  5  0.471641  0.409022
Mean                0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Variance            0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Homogeneity         0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Contrast            0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Entropy             0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Correlation         0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Energy              0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
StdDeviation        0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Skewness            0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Kurtosis            0.000000  0.000000  5.000000e-01  5  0.000000  0.000000
Histogram           0.000000  0.000000  5.000000e-01  5  0.000000  0

/tmp/ipykernel_82189/1038949774.py:49: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  best_model = predictor.get_model_best()
/tmp/ipykernel_82189/1038949774.py:53: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  best_model = predictor.get_model_best()
